In [ ]:
import os
import datetime as dt
import numpy as np
import pandas as pd

import dask.dataframe as dd

import hvplot.dask, hvplot.pandas
import holoviews as hv
from holoviews.streams import Selection1D
from bokeh.models import HoverTool
import panel as pn

import warnings
warnings.simplefilter("ignore")

In [ ]:
if 'TAXI_S3' not in os.environ:
    raise ValueError('Set TAXI_S3 environment variable to an S3 location that you have read/write access to')
taxi_path = os.environ['TAXI_S3']

## Read in data

We'll start by reading in the geo data downloaded from  LINK

In [ ]:
import geopandas as gpd
zones = gpd.read_file('./data/taxi_zones.shp').to_crs('epsg:4326')

zones.hvplot(geo=True)

In [ ]:
import geoviews as gv

basemap = gv.tile_sources.CartoLight()

### Grouped data

Next we'll read in the grouped data that we generated in the exploratory notebooks

In [ ]:
pickup_by_zone_and_time = pd.read_csv("./data/pickup_grouped_by_zone_and_time.csv", index_col="pickup_taxizone_id")
pickup_by_zone_and_time.head()

In [ ]:
zones_dict = dict(zip(zones.LocationID.tolist(), zones.zone.tolist()))

pickup_by_zone = pd.read_csv("./data/pickup_grouped_by_zone.csv", index_col="pickup_taxizone_id")
pickup_by_zone.index = pickup_by_zone.index.map(zones_dict)

dropoff_by_zone = pd.read_csv("./data/dropoff_grouped_by_zone.csv", index_col="dropoff_taxizone_id")
dropoff_by_zone.index = dropoff_by_zone.index.map(zones_dict)

pickup_by_zone.head()

In [ ]:
pickup_by_time = pd.read_csv("data/pickup_grouped_by_time.csv")
pickup_by_time

### Timeseries data

Next we'll read in the hourly timeseries data for the various fields

In [ ]:
tip_timeseries = pd.read_csv("./data/pickup_average_percent_tip_timeseries.csv", index_col="pickup_datetime", parse_dates=True)
ml_prediction_timeseries = pd.read_csv("./data/ml_prediction_timeseries.csv", index_col="pickup_datetime", parse_dates=True)

tip_timeseries.head()

## Construct vizualizations

In this dashboard we'll have three tabs. We'll start with one about volume of rides and aggregate fare, then move on to one about tips and finish with a tab that digests the outputs of the Machine Learning algorithms that we've trained to predict fare.

### Volume tab

In [ ]:
total_rides = pickup_by_zone.total_rides.sum()
total_fare = pickup_by_zone.total_fare.sum()

In [ ]:
volume_intro = """
# Taxi Volume

Ridership by region and average fares for 2017-01-01 to 2020-01-01. 
"""

In [ ]:
logo = pn.pane.SVG("./data/saturn_logo.svg", style={"float": "right"})

In [ ]:
def kpi_box(title, color, value, unit=""):
    if value > 1e9:
        value /= 1e9
        increment = "B"
    elif value > 1e6:
        value /= 1e6
        increment = "M"
    elif value > 1e3:
        value /= 1e3
        increment = "K"
    else:
        increment = ""
    
    return pn.pane.Markdown(
        f"""
        ### {title}
        # {unit}{value :.02f} {increment}
        """,
        style={'background-color': '#F6F6F6', 'border': '2px solid black',
                'border-radius': '5px', 'padding': '10px', 'color': color},
    )

In [ ]:
fares = kpi_box("Total Fares", "#10874a", total_fare, "$")
rides = kpi_box("Total Rides", "#7a41ba", total_rides)
average = kpi_box("Average Fare", "coral", (total_fare / total_rides), "$")

In [ ]:
data = zones.join(pickup_by_zone[["total_rides", "average_fare"]], on="zone")
data["million_rides"] = data.total_rides/1e6

tooltips = [
    ('Total Rides', '@total_rides{(0,0.00 a)}'),
    ('Average Fare', '@{average_fare}{($0.00 a)}'),
    ('Zone', '@zone'),
    ('Borough', '@borough'),
]
hover = HoverTool(tooltips=tooltips)

pickup_map = data.hvplot(
    x="longitude", y="latitude", c="million_rides",
    geo=True, max_width=600, max_height=600,
    alpha=0.6, cmap="viridis", clim=(0, 12), 
    hover_cols=["zone", "borough", "average_fare", "total_rides"], 
    title=f"Rides by pickup location (in Millions)", 
    responsive=True, colorbar=True,
    xaxis=None, yaxis=None, selection_alpha=1).opts(tools=["tap", hover], toolbar="above")

In [ ]:
toggle = pn.widgets.RadioButtonGroup(options=["Pickup", "Dropoff"], value="Pickup")

In [ ]:
@pn.depends(value=toggle)
def volume_table(value):
    data = pickup_by_zone if value == "Pickup" else dropoff_by_zone
    subset = data.total_rides.sort_values(ascending=False)
    subset = subset.loc[subset.index.dropna()]
    
    return pn.Column(
        f"### Top/Bottom 5 {value} Zones", 
        pn.pane.DataFrame(subset.head(5), index_names=False), 
        pn.Spacer(height=10), 
        pn.pane.DataFrame(subset.tail(5), index_names=False, header=False), 
        width_policy="fit"
    )

In [ ]:
data = pickup_by_zone_and_time.copy()
data.index = data.index.map(zones_dict)

def heatmap(C, data=data, **kwargs):
    return data.hvplot.heatmap(
        x="pickup_weekday", 
        y="pickup_hour", 
        C=C,
        hover_cols=["total_rides"] if C == "average_fare" else ["average_fare"],
        xticks=[(0, 'Mon'), (1, 'Tues'), (2, 'Wed'), (3, 'Thur'), (4, 'Fri'), (5, 'Sat'), (6, 'Sun')],
        responsive=True, min_height=500, colorbar=False, **kwargs
    ).opts(toolbar=None, xrotation=90, padding=0)

rides_dmap = heatmap(C="total_rides", groupby="pickup_taxizone_id", cmap="reds")
fare_dmap = heatmap(C="average_fare", groupby="pickup_taxizone_id", cmap="blues")
rides_summary = heatmap(data=pickup_by_time, C="total_rides", cmap="reds", title="Total Rides")
fare_summary = heatmap(data=pickup_by_time, C="average_fare", cmap="blues", title="Average Fare")

volume_heatmap = pn.pane.HoloViews(rides_summary)

In [ ]:
def ride_or_fares_plot(zone, value):
    if value == ["Rides"]:
        if zone is None:
            obj = rides_summary.opts(alpha=1)
        else:
            obj = rides_dmap[zone].opts(title=f"{zone} Rides").opts(alpha=1)
    elif value == ["Fares"]:
        if zone is None:
            obj = fare_summary
        else:
            obj = fare_dmap[zone].opts(title=f"{zone} Fares")
    else:
        if zone is None:
            obj = (fare_summary * rides_summary.opts(alpha=0.5, padding=0)).opts(title="Total Rides/Fares")
        else:
            obj = (fare_dmap[zone] * rides_dmap[zone].opts(alpha=0.5, padding=0)).opts(title=f"{zone}")
    return obj

def on_pickup_tap(index):
    if index:
        zone = zones.loc[index, "zone"].item()
        value = rides_or_fares.value
        volume_heatmap.object = ride_or_fares_plot(zone, value)
    return
       
volume_stream = Selection1D(source=pickup_map)
volume_stream.param.watch_values(on_pickup_tap, ['index']);

In [ ]:
rides_or_fares = pn.widgets.CheckButtonGroup(options=["Rides", "Fares"], value=["Rides"])

In [ ]:
def on_rides_or_fares(target, event):
    index = volume_stream.index
    value = event.new
    if index and value:
        zone = zones.loc[index, "zone"].item()
        volume_heatmap.object = ride_or_fares_plot(zone, value)
    
rides_or_fares.link(volume_heatmap, callbacks={"value": on_rides_or_fares})

def on_reset_heatmap(*args):
    value = rides_or_fares.value
    volume_heatmap.object = ride_or_fares_plot(None, value)

reset_heatmap = pn.widgets.Button(name="Reset")
reset_heatmap.on_click(on_reset_heatmap)

In [ ]:
volume = pn.GridSpec(name="Volume", sizing_mode='stretch_both', min_width=800, min_height=600, max_height=800)
volume[0, :6] = volume_intro
volume[0, 6] = logo
volume[1, 0] = fares
volume[1, 1] = rides
volume[1, 2] = average
volume[1:4, 4:6] = pn.Column(toggle, volume_table)
volume[1:8, 3] = pn.Column(
    pn.pane.Markdown("*Choose rides, fares, or both and select a zone on the map.*", margin=(0, 10)), 
    rides_or_fares, reset_heatmap, volume_heatmap)
volume[2:8, 0:3] = pickup_map * gv.tile_sources.CartoLight()
volume[4:8, 4:6] = "./pie_chart.svg"

## Tip tab

In [ ]:
tip_intro = """
# Analysis of Tips

Tips vary based on time of day, location and many other factors. 
"""

In [ ]:
tip_heatmap = heatmap(data=pickup_by_time, C="average_percent_tip", cmap="coolwarm", clim=(12, 18), title="Average Tip %")

In [ ]:
date_range_slider = pn.widgets.DateRangeSlider(
    name='Show between',
    start=tip_timeseries.index[0], end=tip_timeseries.index[-1],
    value=(pd.Timestamp("2018-01"), pd.Timestamp("2019-02"))
)
discrete_slider = pn.widgets.DiscreteSlider(name='Rolling window', options=['1H', '2H', '4H', '6H', '12H', '1D', '2D', '7D', '14D', '1M'], value='1D')

def tip_plot(xlim, window):
    data = tip_timeseries.rolling(window).mean()
    return data.hvplot(y="percent_tip", xlim=xlim, ylim=(10, 18), responsive=True, min_height=200).opts(toolbar="above")

tip_timeseries_plot = pn.pane.HoloViews(tip_plot(date_range_slider.value, discrete_slider.value))
    
def trim(target, event):
    target.object = tip_plot(event.new, discrete_slider.value)

def roll(target, event):
    target.object = tip_plot(date_range_slider.value, event.new)

discrete_slider.link(tip_timeseries_plot, callbacks={"value": roll})
date_range_slider.link(tip_timeseries_plot, callbacks={"value": trim})

In [ ]:
joined = zones.join(pickup_by_zone, on="zone")

tip_map = joined.hvplot(c="average_percent_tip", geo=True, alpha=0.6, cmap="coolwarm",
                        hover_cols=["zone", "borough"], title="Average Tip %", 
                        clim=(0, 20),responsive=True, colorbar=False,
                        xaxis=None, yaxis=None).opts(toolbar="above")

In [ ]:
tip_table = pickup_by_zone.average_percent_tip.sort_values(ascending=False)
tip_table = tip_table.loc[tip_table.index.dropna()]
tip_pane = pn.Column(
    "### Top/Bottom 5 Tip Zones", 
    pn.pane.DataFrame(tip_table.head(5), header=False, index_names=False),
    pn.Spacer(height=10),
    pn.pane.DataFrame(tip_table.tail(5), header=False, index_names=False),
)

In [ ]:
tips = pn.GridSpec(name="Tips", sizing_mode='stretch_both', min_width=800, min_height=600, max_height=800)

tips[0, :6] = tip_intro
tips[0, 6] = logo
tips[1:5, 0:2] = tip_map * gv.tile_sources.CartoLight()
tips[1:5, 2:4] = tip_pane
tips[1:5, 4:6] = tip_heatmap

tips[5:8, 0:2] = pn.Column(date_range_slider, discrete_slider, "*Use widgets to control rolling window average on the timeseries plot or and to restrict to between certain dates*")
tips[5:8, 2:6] = tip_timeseries_plot

## ML Tab

In [ ]:
ml_intro = """
# Machine Learning

Exploring the historical accuracy of various models. Predict percent tip by consuming deployed models.
"""

In [ ]:
import requests

def tip_prediction(pickup_taxizone_id, dropoff_taxizone_id, datetime, passenger_count):
    try:
        MODEL_URL = os.environ.get('MODEL_URL', 'http://0.0.0.0:8000')
        SCORING_ENDPOINT = f"{MODEL_URL}/api/predict"

        SATURN_TOKEN = os.environ["SATURN_TOKEN"]

        result = requests.post(
            url=SCORING_ENDPOINT,
            json={
                "passenger_count": passenger_count,
                "tpep_pickup_datetime": str(datetime),
                "pickup_taxizone_id": int(pickup_taxizone_id),
                "dropoff_taxizone_id": int(dropoff_taxizone_id)
            },
            headers={
                "Content-Type": "application/json",
                "Authorization": f"token {SATURN_TOKEN}"
            }
        )
    
        return float(result.json()["prediction"]) * 100
    except:
        return -1

In [ ]:
options = {"Choose from map": -1, **{v: k for k, v in zones.zone.to_dict().items()}}
pickup = pn.widgets.Select(name="Pickup", options=options)
dropoff = pn.widgets.Select(name="Dropoff", options=options)
passengers = pn.widgets.IntSlider(name='Passengers', start=0, end=10, step=1, value=2)
plot = zones.hvplot(geo=True, c='zone', legend=False, width=500, height=500, xaxis=None, yaxis=None, alpha=.2, selection_alpha=1).opts(tools=['tap', 'hover'])

def on_map_select(index):
    if index and pickup.value == -1:
        pickup.value = index[0]
    elif index and dropoff.value == -1:
        dropoff.value = index[0]
    return

stream = Selection1D(source=plot)
stream.param.watch_values(on_map_select, ['index'])

overlay = pn.pane.HoloViews(plot * gv.tile_sources.CartoLight())

def on_reset(*args):
    pickup.value = -1
    dropoff.value = -1
    passengers.value = 2
    date.value = dt.datetime.now().date()
    hour.value = 0
    text.background = "#ffffff"
    text.object = None
    stream.update(index=[])
    overlay.object = plot * gv.tile_sources.CartoLight()

reset = pn.widgets.Button(name="Reset", width=80)
reset.on_click(on_reset)

date = pn.widgets.DatePicker(name="Date", value=dt.datetime.now().date())
hour = pn.widgets.DiscreteSlider(
    name="Hour", 
    options=dict(zip(
        ["12am", *[f"{h}am"for h in range(1, 12)] ,"12pm", *[f"{h}pm"for h in range(1, 12)]], 
        list(range(24))
    )))

submit = pn.widgets.Button(name="Predict my tip", button_type='primary', width=200)
text = pn.pane.Markdown(width=200, height=45, style={"padding-left": "10pt"})
helper = pn.pane.Markdown(width=300)

def b(event):
    if pickup.value == -1 or dropoff.value == -1:
        submit.button_type = "danger"
        helper.object = "*You must select pickup and dropoff zone*"
        return
    
    submit.button_type = "primary"
    helper.object = None
    datetime = dt.datetime.combine(date.value, dt.time(hour=hour.value))
    
    prediction = tip_prediction(pickup.value, dropoff.value, datetime, passengers.value)
    
    subset = zones.iloc[[pickup.value, dropoff.value]]
    
    trip = gv.Path((subset.geometry.centroid.x, subset.geometry.centroid.y)).opts(color="black", line_width=2)
    
    obj = plot * gv.tile_sources.CartoLight() * subset.hvplot(geo=True) * trip
    obj.label = f"{subset.zone.tolist()[0]} to {subset.zone.tolist()[1]}"
    
    overlay.object = obj
    
    text.background = "yellow"
    text.object = f"## Prediction: {prediction: .2f}%"
    
submit.on_click(b)

predict = pn.Row(
    pn.Column(
        "## Predict my Tip",
        pickup,
        dropoff, 
        passengers,
        date,
        hour,
        pn.Row(submit, reset),
        helper,
        text,
    ),
    overlay
)

In [ ]:
ml_data =  ml_prediction_timeseries * 100
ml_data = ml_data.rename(columns={"tip_fraction": "actual"})
ml_data.index.name = "time"
ml_timeseries = ml_data.hvplot.line(
    ylabel= "average tip percent",
    min_height=250,
    group_label="Model",
    ylim=(20, 35), 
    xlim=(pd.Timestamp("2019-07-01"), pd.Timestamp("2020-01-01")), legend="right",
).opts(toolbar="left",  legend_offset=(10, 0))

In [ ]:
metrics = dd.read_csv(f"{taxi_path}/ml_results/metrics/*.csv").compute()
metrics_text = """
### Summary of Models

Metrics for all the models.
"""

In [ ]:
ml = pn.GridSpec(name="ML", sizing_mode='stretch_both', min_width=800, min_height=600, max_height=800)

ml[0, :6] = ml_intro
ml[0, 6] = logo
ml[1:3, :6] = ml_timeseries
ml[3:8, 4:6] =pn.Column(metrics_text, pn.pane.DataFrame(metrics, index=False))
ml[3:8, :4] = predict

## Final Dashboard

In [ ]:
pn.Tabs(volume, tips, ml, tabs_location="left").servable(title="Saturn Taxi")

To run this from within Saturn: open a terminal, then cd into the dashboard directory and run `panel serve dashboard.ipynb`. Your dashboard will be at your jupyterlab url with everything from `lab` on replaced with `proxy/5006/dashboard`